In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

# library untuk eksplorasi data
import matplotlib.pyplot as plt
import seaborn as sns

# library untuk membagi data
from sklearn.model_selection import train_test_split

# library untuk evaluasi hasil prediksi
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

# library untuk bebrapa model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.feature_selection import SelectFromModel

# library untuk tunning parameter
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# library untuk menggabungkan beberapa algoritme
from imblearn.pipeline import Pipeline
# library untuk menyeimbangkan data
from imblearn.over_sampling import SMOTE

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from sklearn.calibration import CalibratedClassifierCV

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train_prep1 = df_train.copy()
df_test_prep1 = df_test.copy()

def Pipelining(df_train_prep1,df_test_prep1):
    ################################ Preprocess and Data Cleaning ############################

    ################################ Preprocess and Data Cleaning ############################

    df_train_prep1['Achievement_above_100%_during3quartal'].fillna(0, inplace = True)

    df_train_prep1['GPA'] = np.where(df_train_prep1['GPA']>100,df_train_prep1['GPA']/100,df_train_prep1['GPA'])
    df_train_prep1['GPA'] = np.where(df_train_prep1['Education_level'].isin(['level_0','level_1']),0,df_train_prep1['GPA'])
    df_train_prep1['GPA'] = np.where((df_train_prep1['GPA']==0)&(df_train_prep1['Education_level'] == 'level_4'),3,df_train_prep1['GPA'])

    df_test_prep1['GPA'] = np.where(df_test['GPA']>100,df_test['GPA']/100,df_test['GPA'])
    df_test_prep1['GPA'] = np.where(df_test_prep1['Education_level'].isin(['level_0','level_1']),0,df_test_prep1['GPA'])
    df_test_prep1['GPA'] = np.where((df_test_prep1['GPA']==0)&(df_test_prep1['Education_level'] == 'level_4'),3,df_test_prep1['GPA'])

    ################################ Feature Engineering ############################

    def age_2(x):
        if x <= 35:
            return 'ambis'
        elif x > 35:
            return 'non-ambis'

    def age_3(x):
        if x <= 25:
            return 1
        elif x <= 35:
            return 2
        elif x <= 45:
            return 3
        elif x <= 55:
            return 4
        elif x <= 65:
            return 5

    def year_graduated_2(x):
        if x >= 2015:
            return 'young'
        elif x < 2015:
            return 'old'


    # TOTAL LEAVE 

    df_train_prep1['total leave'] = df_train_prep1['annual leave'] + df_train_prep1['sick_leaves']
    df_test_prep1['total leave'] = df_test_prep1['annual leave'] + df_test_prep1['sick_leaves']

    # AGE

    df_train_prep1['age2'] = 2020 - df_train_prep1['age']
    df_train_prep1['age2'] = df_train_prep1['age2'].apply(age_2)

    df_test_prep1['age2'] = 2020 - df_test_prep1['age']
    df_test_prep1['age2'] = df_test_prep1['age2'].apply(age_2)

    df_train_prep1['age3'] = 2020 - df_train_prep1['age']
    df_train_prep1['age3'] = df_train_prep1['age3'].apply(age_3)

    df_test_prep1['age3'] = 2020 - df_test_prep1['age']
    df_test_prep1['age3'] = df_test_prep1['age3'].apply(age_3)

    # YEAR GRADUATED

    df_train_prep1['year_graduated2'] = df_train_prep1['year_graduated'].apply(year_graduated_2)
    df_test_prep1['year_graduated2'] = df_test_prep1['year_graduated'].apply(year_graduated_2)

    # TOTAL ROTATION

    df_train_prep1['total_rotation'] = df_train_prep1['branch_rotation'] + df_train_prep1['job_rotation'] + df_train_prep1['assign_of_otherposition']
    df_test_prep1['total_rotation'] = df_test_prep1['branch_rotation'] + df_test_prep1['job_rotation'] + df_test_prep1['assign_of_otherposition']

    # IPK O

    df_train_prep1['in study'] = np.where((df_train_prep1['GPA'] == 0)&(df_train_prep1['Education_level'].isin(['level_3','level_4','level_5'])),'study','finish')
    df_test_prep1['in study'] = np.where((df_test_prep1['GPA'] == 0)&(df_test_prep1['Education_level'].isin(['level_3','level_4','level_5'])),'study','finish')

    # LEVEL UP PERSON

    df_train_prep1['person level up'] = np.where(df_train_prep1['job_duration_in_current_job_level']>df_train_prep1['job_duration_in_current_person_level'],'up','stay')
    df_test_prep1['person level up'] = np.where(df_test_prep1['job_duration_in_current_job_level']>df_test_prep1['job_duration_in_current_person_level'],'up','stay')

    # NEW VAR FOR ONE HOT

    df_train_prep1['job_level_oh'] = df_train_prep1['job_level'] 
    df_train_prep1['person_level_oh'] = df_train_prep1['person_level']

    df_test_prep1['job_level_oh'] = df_test_prep1['job_level'] 
    df_test_prep1['person_level_oh'] = df_test_prep1['person_level']
    
    # leave or sick
    
    df_train_prep1['sick_vs_leaves'] = np.where(df_train_prep1['annual leave']>df_train_prep1['sick_leaves'],'leave',np.where(df_train_prep1['annual leave']==df_train_prep1['sick_leaves'],'sama','sick'))
    df_test_prep1['sick_vs_leaves'] = np.where(df_test_prep1['annual leave']>df_test_prep1['sick_leaves'],'leave',np.where(df_test_prep1['annual leave']==df_test_prep1['sick_leaves'],'sama','sick'))

    # age when graduated
    
    df_train_prep1['age when graduated'] = df_train_prep1['year_graduated'] - df_train_prep1['age']
    df_train_prep1['age when graduated2'] = np.where(df_train_prep1['Education_level'].isin(['level_4','level_3']),df_train_prep1['Education_level'].isin(['level_4','level_3'])*df_train_prep1['age when graduated'],0)

    df_test_prep1['age when graduated'] = df_test_prep1['year_graduated'] - df_test_prep1['age']
    df_test_prep1['age when graduated2'] = np.where(df_test_prep1['Education_level'].isin(['level_4','level_3']),df_test_prep1['Education_level'].isin(['level_4','level_3'])*df_test_prep1['age when graduated'],0)
    
    # age per dependencies

    df_train_prep1['age per dependencies'] = (2020-df_train_prep1['age'])/(df_train_prep1['number_of_dependences']+1)
    df_test_prep1['age per dependencies'] = (2020-df_test_prep1['age'])/(df_test_prep1['number_of_dependences']+1)

    # age maried male

    df_train_prep1['age married male'] = (2020-df_train_prep1['age'])*(df_train_prep1['marital_status_maried(Y/N)'] == 'Y')*(df_train_prep1['gender']==1)
    df_test_prep1['age married male'] = (2020-df_test_prep1['age'])*(df_test_prep1['marital_status_maried(Y/N)'] == 'Y')*(df_test_prep1['gender']==1)

    df_train_prep1['papa muda'] = np.where((df_train_prep1['age married male']>=24.9)&(df_train_prep1['age married male']<=32),df_train_prep1['age married male'],0)
    df_test_prep1['papa muda'] = np.where((df_test_prep1['age married male']>=24.9)&(df_test_prep1['age married male']<=32),df_test_prep1['age married male'],0)
    
    # age per dependencies papa muda
    
    df_train_prep1['papa muda per tanggungan'] = df_train_prep1['papa muda']/(df_train_prep1['number_of_dependences']+1)
    df_test_prep1['papa muda per tanggungan'] = df_test_prep1['papa muda']/(df_test_prep1['number_of_dependences']+1)
    
    ############################### Feature Engineering Scikit ############################

    # MODELING

    new_var_one_hot = ['age2','age3','year_graduated2','in study','person level up','sick_vs_leaves']

    one_hot_var = [
        'job_level_oh', 
        'person_level_oh', 
        'Employee_type', 
        'Education_level',
    #     'Employee_status',   
        'gender', 
        'marital_status_maried(Y/N)'] + new_var_one_hot

    ordinal_val = [
        'job_level',
        'person_level',
        'Education_level'
    #     'achievement_target_1', 
    #     'achievement_target_2'
    ]

    new_var_num = ['year_graduated','total leave','total_rotation',
                   'age when graduated','age when graduated2','papa muda']
#                    'papa muda per tanggungan']

    numeric_var = [
        'job_duration_in_current_job_level',   
        'job_duration_in_current_person_level',
        'job_duration_in_current_branch', 
        'age', 
        'number_of_dependences',
        'GPA',   
        'branch_rotation', 
        'job_rotation', 
        'assign_of_otherposition',   
        'annual leave', 
        'sick_leaves',
        'Last_achievement_%', 
        'Achievement_above_100%_during3quartal'] + new_var_num

    # ordinal_mapping = [
    #     {'col':'job_level',
    #     'mapping':{None:0,'JG04':1,'JG05':2,'JG03':3,'JG06':4}},
    #     {'col':'person_level',
    #     'mapping':{None:0,'PG01':1,'PG02':2,'PG03':3,'PG04':4, 'PG05':5, 'PG06':6, 'PG07':7, 'PG08':8}},
    #     {'col':'Education_level',
    #     'mapping':{None:0,'level_0':1,'level_1':2,'level_2':3, 'level_3':4, 'level_4':5, 'level_5':6}}    
    #     ]

    # ordinal_encoder = ce.OrdinalEncoder(cols = ['job_level','person_level','Education_level'],mapping = ordinal_mapping)

    one_hot_encoder_pipeline = Pipeline([
                                        ('one hot encoder',OneHotEncoder(handle_unknown = 'ignore'))
    ])

    # numerical_pipeline = SimpleImputer(strategy = 'constant',fill_value = 0)

    numerical_pipeline = Pipeline([
                                    ('imputer',SimpleImputer(strategy = 'constant',fill_value = 0)),
                                    ('bin',KBinsDiscretizer(encode = 'onehot',strategy = 'kmeans',n_bins=5))
    ])

    transformer = ColumnTransformer([
        ('one hot encoder',one_hot_encoder_pipeline,one_hot_var),
    #     ('ordinal encoder',ordinal_encoder,ordinal_val),
        ('numerical_pipeline',numerical_pipeline,numeric_var)
    ])

    ################################ Daata Splitting ############################

    var_x = numeric_var + one_hot_var #+ ordinal_val

    # data to return
    X = df_train_prep1[var_x]
    X_sub = df_test_prep1[var_x]
    y = df_train_prep1['Best Performance']


    ################################ Sklearn Prepreocess Fitting ############################

    # feature names
    transformer.fit(X) # fitting only for feature namse
    one_hot_result = list(transformer.transformers_[0][1]['one hot encoder'].get_feature_names())
    features = one_hot_result + numeric_var
    
    return transformer, features, X, y, X_sub

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
transformer, features, X, y, X_sub = Pipelining(df_train_prep1,df_test_prep1)

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:189: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 12 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


In [3]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X,y, stratify = y, test_size = 2000, random_state = 7062021)
X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, stratify = y_trainval, test_size = 2000, random_state = 7062021)

In [4]:
model = LogisticRegression(solver = 'newton-cg')
select = SelectFromModel(model, threshold="median")

estimator = Pipeline([
                      ('preprocess',transformer),
#                       ('select',select),
                      ('clf',model)
])

In [5]:
estimator.fit(X_train, y_train)

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:189: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('one hot encoder',
                                                  Pipeline(steps=[('one hot '
                                                                   'encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job_level_oh',
                                                   'person_level_oh',
                                                   'Employee_type',
                                                   'Education_level', 'gender',
                                                   'marital_status_maried(Y/N)',
                                                   'age2', 'age3',
                                                   'year_graduated2',
                                                   'in study',
                                                   'person level up'

In [6]:
y_prob_test = estimator.predict_proba(X_test)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_test,y_prob_test),'\n')
print('classification report \n', classification_report(y_test,y_pred_test))

auc 0.5953038894338509 

classification report 
               precision    recall  f1-score   support

           0       0.88      0.45      0.59      1706
           1       0.17      0.66      0.27       294

    accuracy                           0.48      2000
   macro avg       0.53      0.55      0.43      2000
weighted avg       0.78      0.48      0.55      2000



In [7]:
y_prob_test = estimator.predict_proba(X_train)[:,1]
y_pred_test = np.where(y_prob_test > 0.13, 1, 0)

print('auc',roc_auc_score(y_train,y_prob_test),'\n')
print('classification report \n', classification_report(y_train,y_pred_test))

auc 0.6203313748897887 

classification report 
               precision    recall  f1-score   support

           0       0.90      0.46      0.61      6103
           1       0.18      0.71      0.29      1050

    accuracy                           0.49      7153
   macro avg       0.54      0.58      0.45      7153
weighted avg       0.80      0.49      0.56      7153



# Prediction

In [ ]:
estimator.fit(X,y)
y_pred_sub = estimator.predict_proba(X_sub)[:,1]
df_test['Best Performance'] = y_pred_sub
df_submission = df_test[['Best Performance']]
df_submission.reset_index().to_csv('submission.csv',index = False)

C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:189: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
C:\Users\muhyi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 12 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
